# ETL del dataset "australian_users_items.json"

En este archivo de iniciaremos con la Extracción, transformación y carga de los datos. El objetivo es extraer el contenido de un JSON para visualizar los datos a través de un dataFrame de Pandas y luego guardarlos en un formato CSV. 

In [1]:
import pandas as pd
import ast
import tools
import warnings
warnings.filterwarnings("ignore")


ruta_items = "australian_users_items.json"

# Se lee de cada línea del dataset
filas_items = []
with open(ruta_items, "r", encoding="utf-8") as f:
    for line in f.readlines():
        filas_items.append(ast.literal_eval(line))

# Se convierte en dataframe
df_users_items = pd.DataFrame(filas_items)
df_users_items


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Verificaremos qué tipo de datos contiene

In [2]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [3]:
df_users_items.describe()

,items_count
count,88310.000000
mean,58.353629
std,122.312095
min,0.000000
25%,3.000000
50%,26.000000
75%,73.000000
max,7762.000000


In [4]:
tools.ver_tipo_datos(df_users_items)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,items_count,[<class 'int'>],100.0,0.0,0
2,steam_id,[<class 'str'>],100.0,0.0,0
3,user_url,[<class 'str'>],100.0,0.0,0
4,items,[<class 'list'>],100.0,0.0,0


## Limpieza y transformación de datos 
Observamos que el dataframe no contiene valores nulos. Sin embargo, la columna "items" es una lista, así que vamos a explorarla para determinar cuál es su estructura. 


In [5]:
df_users_items["items"][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

Como podemos observar, la columna "items" se encuentra anidada, ya que es una lista de diccionarios. Por lo tanto, vamos a normalizarla de manera que obtengamos una columna por cada clave de ese diccionario, manteniendo la trazabilidad de las columnas "steam_id","items_count","user_id" y "user_url".

In [6]:
# Normalizamos la columna "items"
df_users_items = pd.json_normalize(filas_items, record_path=["items"], 
                                   meta=["steam_id","items_count","user_id", "user_url"])

df_users_items

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [7]:
tools.ver_tipo_datos(df_users_items)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,item_id,[<class 'str'>],100.0,0.0,0
1,item_name,[<class 'str'>],100.0,0.0,0
2,playtime_forever,[<class 'int'>],100.0,0.0,0
3,playtime_2weeks,[<class 'int'>],100.0,0.0,0
4,steam_id,[<class 'str'>],100.0,0.0,0
5,items_count,[<class 'int'>],100.0,0.0,0
6,user_id,[<class 'str'>],100.0,0.0,0
7,user_url,[<class 'str'>],100.0,0.0,0


In [8]:
# Buscaremos los registros duplicados para posteriormente eliminarlos

duplicados = df_users_items[df_users_items.duplicated(subset=["steam_id","item_id","playtime_forever"])]
duplicados.shape

(59117, 8)

In [9]:
df_users_items = df_users_items.drop_duplicates(keep="first")
df_users_items

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [10]:
# Verificamos los datos dentro de la columna "playtime_forever"
max_valor = df_users_items["playtime_forever"].max()
print(f"El valor máximo en la columna 'playtime_forever' es: {max_valor}")

El valor máximo en la columna 'playtime_forever' es: 642773


Observamos que los datos están en minutos, vamos a pasarlos a hora y los redondearemos a 2 decimales.

In [11]:
df_users_items["played_hours"] = (df_users_items["playtime_forever"] / 60).round(2)


In [12]:
# Los 5 valores máximos
max_5 = df_users_items.nlargest(5, "played_hours")

# Los 5 valores mínimos
min_5 = df_users_items.nsmallest(5, "played_hours")

print("Top 5 valores máximos:")
print(max_5[["played_hours"]])

print("\nTop 5 valores mínimos:")
print(min_5[["played_hours"]])

Top 5 valores máximos:
         played_hours
587715       10712.88
2499068      10588.25
4075729      10540.87
1495340      10223.52
1836985      10001.13

Top 5 valores mínimos:
   played_hours
1           0.0
3           0.0
4           0.0
5           0.0
6           0.0


Observamos que hay una valores 0 en la columna "played_hours", buscaremos cuantos valores así hay. 

In [13]:
filas_con_cero = (df_users_items["played_hours"] == 0).sum()

print(f"Existen {filas_con_cero} filas con valor 0 en la columna ´played_hours´")

Existen 1847730 filas con valor 0 en la columna ´played_hours´


Efectivamente hay 1847730 registros con valor 0 en la columna "played_hours". Eliminaremos los registros con este valor en dicha columna, porque no serán útiles para nuestro estudio propuesto

In [14]:
df_users_items = df_users_items[df_users_items["played_hours"] != 0]
df_users_items

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url,played_hours
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.10
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.12
8,300,Day of Defeat: Source,4733,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,78.88
9,240,Counter-Strike: Source,1853,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30.88
10,3830,Psychonauts,333,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,5.55
...,...,...,...,...,...,...,...,...,...
5153202,304930,Unturned,677,677,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,11.28
5153203,227940,Heroes & Generals,43,43,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.72
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.05
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.07


También, eliminaremos las columnas que no serán relevantes para nuestro estudio.

In [15]:
df_users_items = df_users_items.drop(["playtime_2weeks","steam_id","playtime_forever"], axis=1)

df_users_items

,item_id,item_name,items_count,user_id,user_url,played_hours
0,10,Counter-Strike,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.10
2,30,Day of Defeat,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.12
8,300,Day of Defeat: Source,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,78.88
9,240,Counter-Strike: Source,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30.88
10,3830,Psychonauts,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,5.55
...,...,...,...,...,...,...
5153202,304930,Unturned,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,11.28
5153203,227940,Heroes & Generals,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.72
5153206,388490,One Way To Die: Steam Edition,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.05
5153207,521570,You Have 10 Seconds 2,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.07


In [16]:
df_users_items

,item_id,item_name,items_count,user_id,user_url,played_hours
0,10,Counter-Strike,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.10
2,30,Day of Defeat,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.12
8,300,Day of Defeat: Source,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,78.88
9,240,Counter-Strike: Source,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30.88
10,3830,Psychonauts,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,5.55
...,...,...,...,...,...,...
5153202,304930,Unturned,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,11.28
5153203,227940,Heroes & Generals,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.72
5153206,388490,One Way To Die: Steam Edition,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.05
5153207,521570,You Have 10 Seconds 2,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.07


### Carga del conjunto de datos

Guardaremos el conjunto de datos transformado como users_items_cleaned.csv


In [17]:
df_users_items.to_csv("users_items_cleaned.csv", index= False, encoding="utf-8")
print(f"Se guardó el archivo")

Se guardó el archivo
